In [33]:
# ETL Project
import csv
from datetime import datetime
import json

# Read CSV files
def read_csv(file_path):
        with open(file_path, 'r') as file:
            reader = csv.DictReader(file)
            return list(reader)

salesData = read_csv("/content/sample_data/sales(in).csv")
productData = read_csv("/content/sample_data/Product(in).csv")
customerData = read_csv("/content/sample_data/Customer(in).csv")

# Clean sales data
def cleaned_sales_data(salesData):
    data_seen = set()
    cleaned_data = []
    for sale in salesData:
        sale_id = sale.get('sale_id')
        if sale_id and sale_id not in data_seen and all(sale.values()):
            data_seen.add(sale_id)
            cleaned_data.append(sale)
    return cleaned_data

# Integrate data
def integrate_data(cleaned_sales_data, productData, customerData):
    product_map = {product['product_id']: product for product in productData if 'product_id' in product}
    customer_map = {customer['customer_id']: customer for customer in customerData if 'customer_id' in customer}
    combined_data = []

    for sale in cleaned_sales_data:
        product = product_map.get(sale.get('product_id'))
        customer = customer_map.get(sale.get('customer_id'))
        if product and customer:
                combined_data.append({
                    'sale_id': sale['sale_id'],
                    'sale_date': sale['date'],
                    'customer_id': sale['customer_id'],
                    'product_id': sale['product_id'],
                    'quantity': int(sale['quantity']),
                    'price': float(sale['price']),
                    'customer_name': customer.get('name', ''),
                    'customer_email': customer.get('email', ''),
                    'product_name': product.get('product_name', ''),
                    'product_category': product.get('category', ''),
                    'product_price': float(product.get('cost_price', 0)),
                    'total_amount': int(sale['quantity']) * float(sale['price']),
                    'day_of_week': datetime.strptime(sale['date'], '%m/%d/%Y').strftime('%A'),
                    'month': datetime.strptime(sale['date'], '%m/%d/%Y').strftime('%B'),
                    'quarter': (int(datetime.strptime(sale['date'], '%m/%d/%Y').month) - 1) // 3 + 1
                })
    return combined_data

# Execute ETL
cleaned_sales = cleaned_sales_data(salesData)
transformed_data = integrate_data(cleaned_sales, productData, customerData)

# Write transformed data to a new CSV file
output_file_path = "/content/sample_data/transformed_datafile.csv"
with open(output_file_path, "w", newline='') as file:
    writer = csv.DictWriter(file, [
        'sale_id', 'sale_date', 'customer_id', 'product_id', 'quantity', 'price',
        'customer_name', 'customer_email', 'product_name', 'product_category',
        'product_price', 'total_amount', 'day_of_week', 'month', 'quarter'
    ])
    writer.writeheader()
    writer.writerows(transformed_data)
print(f"Combined data written to {output_file_path}")

# Data Analysis
# Calculate total sales and profit by product category.
sales_by_category = {}
for sale in transformed_data:
    category = sale['product_category']
    if category not in sales_by_category:
        sales_by_category[category] = 0
    sales_by_category[category] += sale['total_amount']

# Identify top-selling products and key customers.
top_categories = sorted(sales_by_category.items(), key=lambda x: x[1], reverse=True)[:5]
# Accumulate sales by customer
key_customers = {}
for sale in transformed_data:
    customer_id = sale.get('customer_id')  # Use `.get()` for safer key access
    total_amount = sale.get('total_amount', 0)
    if customer_id:  # Ensure valid customer_id
        key_customers[customer_id] = key_customers.get(customer_id, 0) + total_amount

# Identify Top 5 Customers
sorted_customers = sorted(key_customers.items(), key=lambda x: x[1], reverse=True)
top_customers = sorted_customers[:5]
key_customers = {customer[0]: customer[1] for customer in top_customers}


# Analyze sales trends over time (daily, monthly, quarterly).
daily_sales = {}
for sale in transformed_data:
    day = sale['day_of_week']
    if day not in daily_sales:
        daily_sales[day] = 0
    daily_sales[day] += sale['total_amount']
monthly_sales = {}
for sale in transformed_data:
    month = sale['month']
    if month not in monthly_sales:
        monthly_sales[month] = 0
    monthly_sales[month] += sale['total_amount']
quarterly_sales = {}
for sale in transformed_data:
    quarter = sale['quarter']
    if quarter not in quarterly_sales:
        quarterly_sales[quarter] = 0
    quarterly_sales[quarter] += sale['total_amount']

# Generate summary report
def generate_summary_report(sales_by_category, top_categories, monthly_sales):
    analysis_report = {
        'sales_by_category': sales_by_category,
        'top_categories': top_categories,
        'key_customers': key_customers,
        'daily_sales': daily_sales,
        'monthly_sales': monthly_sales,
        'quarterly_sales': quarterly_sales
    }
    return json.dumps(analysis_report, indent=4)

summary = generate_summary_report(sales_by_category, top_categories, monthly_sales)

# Write summary report to a file
summary_file_path = "/content/sample_data/sales_report.txt"
with open(summary_file_path, 'w') as file:
    file.write(summary)
print(f"Summary report generated and saved to {summary_file_path}")
print(summary)

Combined data written to /content/sample_data/transformed_datafile.csv
Summary report generated and saved to /content/sample_data/sales_report.txt
{
    "sales_by_category": {
        "Electronics": 12228.13999999999,
        "Home & Office": 4649.069999999999,
        "Stationery": 1439.1000000000004
    },
    "top_categories": [
        [
            "Electronics",
            12228.13999999999
        ],
        [
            "Home & Office",
            4649.069999999999
        ],
        [
            "Stationery",
            1439.1000000000004
        ]
    ],
    "key_customers": {
        "C123": 999.9,
        "C111": 799.92,
        "C107": 599.9399999999999,
        "C119": 599.9399999999999,
        "C103": 547.92
    },
    "daily_sales": {
        "Sunday": 2561.49,
        "Monday": 2671.4399999999996,
        "Tuesday": 2269.5099999999998,
        "Wednesday": 2827.4399999999996,
        "Thursday": 2559.47,
        "Friday": 2461.49,
        "Saturday": 2965.4699999